# BioExplorer - Vasculature
![](../bioexplorer_vasculature_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load vasculature

In [ ]:
assembly_name = 'Vasculature'
population_name = 'vasculature'
be.remove_assembly(assembly_name)
vasculature_assembly = be.add_assembly(assembly_name)
vasculature_model = be.add_vasculature(
    assembly_name=assembly_name,
    population_name=population_name,
    use_sdf=True,
    quality=be.GEOMETRY_QUALITY_MEDIUM,
    sql_filter='radius>3.5'
)

In [ ]:
be.reset_camera()
core.set_renderer()

In [ ]:
be.set_vasculature_color_scheme(
    assembly_name=assembly_name,
    color_scheme=be.VASCULATURE_COLOR_SCHEME_NONE,
    palette_name='Reds'
)
core.set_renderer()

In [ ]:
vasculature_info = be.get_vasculature_info(assembly_name=assembly_name)
print(vasculature_info)

In [ ]:
status = core.set_renderer(
    current='bio_explorer',
    subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 1000.0
params.shadows = 1.0
params.soft_shadows = 1.0
status = core.set_renderer_params(params)